## Instrucciones: correr el código sólo con los datos de secuestro

In [2]:
import numpy as np
import pandas as pd
import random

### Creación de la clase Optimizador

In [20]:
class Optimizador:
    #siempre empezar con _init_ (dos guiones)
    def __init__(self, f, max_iter, eps): #parámetros globales de la clase. Siempre empezar con self
        #toma h por default sin que se lo mandes, pero f no
        self.f  = f
        self.max_iter = max_iter
        self.eps = eps
        
    #def repr hace que si le das print(Derivadas) te define qué mostrar
    def __repr__(self):
        return "Los parámetros globales de la clase son: \n f = %s \n eps = %s \n max_iters = %s" % (self.f, self.eps, self.max_iters)

    def grad(self, xk):
        n   = xk.size
        res = np.zeros(n)
        for i in range(n):
            aux = np.zeros(n)
            aux[i] = self.eps
            x1 = xk + aux
            res[i] = (self.f(x1) - self.f(xk))/self.eps        
        return res
    
    def hess(self,xk):
        n = xk.size
        res = np.zeros((n,n))
        for i in range(n):
            aux = np.zeros(n)
            aux[i] = self.eps
            x1 = xk + aux
            res[:,i] = (self.grad(x1)-self.grad(xk))/self.eps
        return res
    
    def alfa(self,xk,p): #Para que alfa satisfaga las condiciones de Wolfe
        a_gorro = 1
        ro = 0.8
        c = .001
        alpha = a_gorro
        while self.f(xk+alpha*p) > self.f(xk)+c*alpha*(self.grad(xk).T).dot(p):
            alpha = ro*alpha
        return alpha
    
    def calcula_gamma(self,A): #Para poder modificar la Hessiana sumándole un matriz múltiplo de la identidad
        k = 100
        n = A.shape[0]
        beta = 0.001
        diagonal = np.diagonal(A)
        minimo = np.amin(diagonal)
        if minimo > 0:
            gamma = 0
        else:
            gamma = -minimo + beta
        #end if

        for i in range(k):     
            try:
                np.linalg.cholesky(A+gamma*np.identity(n))
            except np.linalg.LinAlgError:
                gamma = max([2*gamma, beta])
            else:
                break


        return gamma
    
    def NEWTON(self,xk): #Usamos la dirección de Newton para generar el direcció  pk
        for i in range(self.max_iter):
            H  = self.hess(xk)
            pk = -np.linalg.inv(H).dot(self.grad(xk))
            xk = xk + pk
        return xk
    
    def NEWTON_MOD(self,xk): #Usamos una modificación a la Hessiana para generar la dirección pk
        n = xk.size
        for i in range(self.max_iter):
            gamma = self.calcula_gamma(self.hess(xk))
            Bk = self.hess(xk)+gamma*np.identity(n)

            pk = -np.linalg.inv(Bk).dot(self.grad(xk))        
            alpha = self.alfa(xk,pk)
            xk = xk+alpha*pk
            #print(f(x0)) (para ver si la función realmente decrece)
        return xk
    
    def BFGS(self,xk): #En lugar de calcular la Hessiana en cada paso, calculamos la matriz H_k+1 en terminos de H_k
        n = xk.size
        I = np.identity(n)
        Hk = I
        while np.linalg.norm(self.grad(xk)) > self.eps:
            pk = -Hk.dot(self.grad(xk))
            alfa_k = self.alfa(xk,pk)
            aux = xk
            xk = xk + alfa_k*pk
            sk = xk - aux
            yk = self.grad(xk)-self.grad(aux)
            aux2 = float(yk.dot(sk))
            #Agregamos este if porque por alguna razón que desconocemos el producto punto se hace cero antes de que la norma
            #del gradiente evaluada en xk sea menor que epsilon
            if aux2 == 0:
                break
            else:
                rho_k = 1.0/aux2
                Hk = ((I - rho_k*sk.dot(yk.T)).dot(Hk)).dot(I - rho_k*yk.dot(sk.T)) + rho_k*(sk.dot(sk.T))
            print(xk)
        return xk
    
    def Optimiza(self, metodo, xk):
        if metodo == "BFGS":
            resp = self.BFGS(xk)
        elif metodo == "NEWTON_MOD":
            resp = self.NEWTON_MOD(xk)
        elif metodo == "NEWTON":
            resp = self.NEWTON(xk)
        else:
            resp = "Método no encontrado"
        return resp

### Prueba en la función de Rosenbrock

In [4]:
def rosenbrock(xk):
    a = 1
    b = 100
    return (a-xk[0])**2+b*(xk[1]-xk[0]**2)**2

El primer objeto tiene como atributos a la función de rosenbrock, con un número de 1000 iteraciones para las funciones que así lo requieran y con un valor de toleracia (epsilon) de 0.00001

In [16]:
objeto_1 = Optimizador(rosenbrock,1000,0.00001)

Optimizamos la función de **rosenbrock** con el Algoritmo de Newton:

In [6]:
objeto_1.Optimiza("NEWTON",np.array([2.5,8]))

array([0.99700991, 0.99402376])

Optimizamos la función de **rosenbrock** con el método Búsqueda Lineal con Modificación a la Hessiana:

In [7]:
objeto_1.Optimiza("NEWTON_MOD",np.array([2.5,8]))

array([0.99968429, 0.9993484 ])

Optimizamos la función de **rosenbrock** con el método BFGS:

In [17]:
objeto_1.Optimiza("BFGS",np.array([2.5,8]))

array([1.00394696, 1.00796915])

Si llamamos a la función "Optimiza" y elegimos un método que no pertenece a la clase entonces:

In [21]:
objeto_1.Optimiza("kdjvbdfkv",np.array([1.2,1.5]))

'Método no encontrado'

### Prueba en la función de costos

In [23]:
#ENTRADAS:
# xk ==> es un vector con un número par de entradas en donde la primera mitad de las entradas
#representa la latitud de las cámaras y la segunda mitad de las entradas representa su longitud
def costo(xk):
    #Para convertir al vector xk en la matriz camaras
    n = xk.size
    c = 0
    vec1 = np.zeros(int(n/2))
    vec2 = np.zeros(int(n/2))
    for i in range(0,int(n/2)):
        vec1[i] = xk[i]
        vec2[i] = xk[int(n/2) + i]
    camaras = np.array([vec1,vec2])
    camaras = camaras.T
    #Para leer los datos de los crimenes
    datos = pd.read_csv('crime_data.csv')
    datos = datos.to_numpy()
    #Para convertir los datos en una matriz
    latitudes = datos[:,3]
    longitudes = datos[:,4]
    crimenes = np.array([latitudes,longitudes])
    crimenes = crimenes.T
    #Ahora ya tenemos a una matriz de crimenes y una matriz de camaras
    for i in range(len(camaras)):
        for j in range(len(crimenes)):
            c += np.linalg.norm(camaras[i] - crimenes[j])**2
    for i in range(len(camaras)):
        for j in range(len(camaras)):
            if i != j:
                c += 1/(0.00001 + (np.linalg.norm(camaras[i]-camaras[j]))**2)
    return c

In [9]:
#Generamos un vector aleatorio de latitudes y longitudes de cada cámara para poder evaluar a nuestra
#función de costos en un valor x0 que sea razonable
x_camaras = np.zeros(10) 
for i in range(5):
    x_camaras[i] = 19 + np.random.rand()
    x_camaras[i+5] = -100 + np.random.rand()
print(x_camaras)

[ 19.58794971  19.54102211  19.94941456  19.86666599  19.94379121
 -99.09407951 -99.0590898  -99.54159422 -99.30066669 -99.50789578]


In [10]:
#Evaluamos la función de costos en el vector aleatorio que generamos
resp = costo(x_camaras)
print(resp)

2448.786085975055


El segundo objeto tiene como atributos a nuestra función  de costo, con un número de 50 iteraciones para las funciones que lo requieran y con un valor de tolerancia (épsilon) de 0.0001

In [24]:
objeto_2 = Optimizador(costo,50,0.001)

2.1) Optimizamos la función **costo** con el Algoritmo de Newton:

In [12]:
x_newton = objeto_2.Optimiza("NEWTON",x_camaras)
resp1 = costo(x_newton)
print("El mínimo de la función está cerca de:", x_newton)
print("El valor de la función costo en el xk encontrado:", resp1)

El mínimo de la función está cerca de: [  19.33883429   19.26891787   19.54610657   19.40378261   19.47047433
  -98.72211675  -98.22682265 -100.09465947  -99.16671208  -99.60845164]
El valor de la función costo en el xk encontrado: 93.10484666280433


3.1) Optimizamos a la función **costo** con modificación a ala Hessiana:

In [13]:
x_newton_mod = objeto_2.Optimiza("NEWTON_MOD",x_camaras)
resp2 = costo(x_newton_mod)
print("El mínimo de la función está cerca de:", x_newton_mod)
print("El valor de la función costo en el xk encontrado:", resp2)

El mínimo de la función está cerca de: [ 19.36936018  18.74121481  19.48008881  19.34715427  19.98358397
 -98.50881613 -99.12797648 -99.7610375  -99.15487474 -99.05535062]
El valor de la función costo en el xk encontrado: 67.30183515791505


4.1) Optimizamos a la función **costo** con el algoritmo BFGS:

In [25]:
x_bfgs = objeto_2.Optimiza("BFGS",x_camaras)
resp3 = costo(x_bfgs)
print("El mínimo de la función está cerca de:", x_bfgs)
print("El valor de la función costo en el xk encontrado:", resp3)

[  20.43284746   18.65687385   20.90608934   19.86111886   19.13891523
  -99.73817728  -98.41463556 -105.08365247  -99.27414115  -94.44856149]
[  18.19480181   17.02028962   18.41039767   17.70758435   17.32766663
 -101.46578979 -100.58651063 -104.6807876  -101.06979235  -98.16385267]
[  21.21136837   20.03702681   21.4268323    20.72415013   20.34436679
  -98.44908283  -97.56992984 -101.66322883  -98.05308228  -95.14788935]
[  19.05974515   17.88540604   19.27520718   18.57252689   18.19274549
 -100.60070403  -99.72155286 -103.81483776 -100.20470344  -97.29952126]
[  20.82912228   19.6547832    21.04458429   20.34190402   19.96212264
  -98.83132688  -97.95217573 -102.04546046  -98.43532628  -95.53014423]
[  19.57958613   18.40524706   19.79504814   19.09236788   18.7125865
 -100.08086302  -99.20171187 -103.2949966   -99.68486243  -96.77968038]
[  20.51320454   19.33886546   20.72866655   20.02598629   19.64620491
  -99.14724462  -98.26809346 -102.3613782   -98.75124402  -95.84606197]


KeyboardInterrupt: 